# Setup Development Environment

This notebook walks you through the steps needed to setup your development environment for Oracle Functions.

## Prerequisites

The following software **must** be installed:

1. Docker
1. Python 3


## Step 1 - Install the OCI Command Line Interface (CLI)

The following command installs the OCI CLI with the all the defaults:

In [ ]:
bash -c "$(curl -L https://raw.githubusercontent.com/oracle/oci-cli/master/scripts/install/install.sh)" -- --accept-all-defaults

## Step 2 - Install Fn CLI


In [ ]:
!curl -LSs https://raw.githubusercontent.com/fnproject/cli/master/install | sh